In [1]:
from enum import Enum
import json
import os
from typing import Optional, Dict, Any, List

import outlines
from outlines import models, generate
import pandas as pd
import pprint
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


# 1. Prepare Input

In [2]:
SYSTEM_MESSAGE = '''You are building an emotion‑prediction component that, given a character name and a brief situational description, must identify the character’s emotional state and the reason for it.
When given an input of the form:
```
Character:  {character}
Source: {source}
```
your job is to:

1. **Map the Emotion:**  
   - Choose among the 8 primary emotions from Plutchik’s Wheel (joy, trust, fear, surprise, sadness, disgust, anger, anticipation).  
   - For each, assign one of: `"na"` (not applicable), `"low"`, `"medium"`, or `"high"`.

2. **Write a Reason:**  
   - Provide a single sentence explaining why the character feels as you’ve labeled.

## Map the Emotion: Map the target emotion onto the 8 primary emotions from Plutchik’s Wheel
* "joy": Joy is a bright, uplifting emotion that reflects happiness, satisfaction, and a sense of well-being. It often arises when our desires are fulfilled or we experience positive moments, and it helps energize both our minds and bodies. Joy can enhance social connections and overall resilience by radiating positivity
* "trust": Trust is the reassuring feeling of confidence and security in another person or situation. It builds from consistent, reliable interactions and underpins strong, supportive relationships. This emotion fosters cooperation and reduces anxiety by creating a sense of safety
* "fear": Fear is an instinctive response to perceived threats that activates our fight-or-flight mechanism. It heightens awareness and prepares our body to respond quickly to danger, making it essential for survival. Despite its discomfort, fear is a crucial signal that prompts protective action and risk assessment
* "surprise": Surprise occurs when we encounter the unexpected, momentarily halting our regular thought process. This emotion can be positive, neutral, or even negative, depending on the context, and often sparks curiosity about what comes next. Its brief nature helps redirect our focus and encourages adaptive responses to new situations
* "sadness": Sadness is a deep, reflective emotion that often emerges from loss, disappointment, or unmet expectations. It can lead to introspection and a desire for support as we navigate feelings of grief or dejection. Although challenging, sadness can also foster empathy and pave the way for emotional healing and growth
* "disgust": Disgust is an aversive emotion that signals rejection toward something perceived as harmful, unclean, or morally offensive. It serves as a protective mechanism, prompting us to avoid substances or situations that might be dangerous. This emotion plays a vital role in maintaining both physical health and ethical boundaries
* "anger": Anger arises when we perceive injustice, frustration, or a threat to our well-being, often urging us to act in response. It can manifest as physical tension and heightened energy, signaling that something in our environment needs to change. When managed effectively, anger can motivate constructive action and help assert personal boundaries
* "anticipation": Anticipation is the forward-looking emotion characterized by a mix of excitement and apprehension about future events. It motivates preparation and planning while balancing hope with cautious vigilance. This emotion bridges the gap between our present state and the potential for positive outcomes in the future

For each emotion, assign one of the following intensities:
* "na" (not applicable)
* "low"
* "medium"
* "high"

## Write a Reason:
Provide a one-sentence rationale ("reason") explaining why the subject (if xReact) or the other person (if oReact) feels the given emotion(s).
ex. “She feels empowered and confident after cutting out social media.”

Return in the following JSON format (no extra keys, no explanation outside the JSON)
{
    "emotion": {
        "joy": "na" | "low" | "medium" | "high",
        "trust": "na" | "low" | "medium" | "high",
        "fear": "na" | "low" | "medium" | "high",
        "surprise": "na" | "low" | "medium" | "high",
        "sadness": "na" | "low" | "medium" | "high",
        "disgust": "na" | "low" | "medium" | "high",
        "anger": "na" | "low" | "medium" | "high",
        "anticipation": "na" | "low" | "medium" | "high"
    },
    "reason": "One sentence explaining why these emotions occur"
}
Only return the JSON'''

USER_TEMPLATE = '''Source: {source}
Character: {character}'''

class RelationshipStatus(str, Enum):
    na = "na"
    low = "low"
    medium = "medium"
    high = "high"
    
class EmotionLabel(BaseModel):
    joy: RelationshipStatus
    trust: RelationshipStatus
    fear: RelationshipStatus
    surprise: RelationshipStatus
    sadness: RelationshipStatus
    disgust: RelationshipStatus
    anger: RelationshipStatus
    anticipation: RelationshipStatus
    
    # class Config:
    #     extra = Extra.forbid
    #     use_enum_values = True
        
class EntryResult(BaseModel):
    emotion: EmotionLabel
    reason: str

In [3]:
llm_model = "gpt-4.1-mini-2025-04-14"
df = pd.read_csv(f"data/comet/test_{llm_model}.tsv", sep="\t")
print(df.shape, df.columns)

(4452, 16) Index(['uid', 'original_idx', 'original_src', 'original_relation',
       'original_tgt', 'source', 'character', 'joy', 'trust', 'fear',
       'surprise', 'sadness', 'disgust', 'anger', 'anticipation', 'reason'],
      dtype='object')


In [4]:
row = df.iloc[1]
user_message = USER_TEMPLATE.format(
    source=row['source'],
    character=row['character']
)
assistant_message = json.dumps(
    {
        "emotion": {
            "joy": row['joy'],
            "trust": row['trust'],
            "fear": row['fear'],
            "surprise": row['surprise'],
            "sadness": row['sadness'],
            "disgust": row['disgust'],
            "anger": row['anger'],
            "anticipation": row['anticipation']
        },
        "reason": row['reason']
    }
)

messages = [
    {"role": "system", "content": SYSTEM_MESSAGE},
    {"role": "user", "content": user_message},
    {"role": "assistant", "content": assistant_message}
]

In [5]:
messages

[{'role': 'system',
  'content': 'You are building an emotion‑prediction component that, given a character name and a brief situational description, must identify the character’s emotional state and the reason for it.\nWhen given an input of the form:\n```\nCharacter:  {character}\nSource: {source}\n```\nyour job is to:\n\n1. **Map the Emotion:**  \n   - Choose among the 8 primary emotions from Plutchik’s Wheel (joy, trust, fear, surprise, sadness, disgust, anger, anticipation).  \n   - For each, assign one of: `"na"` (not applicable), `"low"`, `"medium"`, or `"high"`.\n\n2. **Write a Reason:**  \n   - Provide a single sentence explaining why the character feels as you’ve labeled.\n\n## Map the Emotion: Map the target emotion onto the 8 primary emotions from Plutchik’s Wheel\n* "joy": Joy is a bright, uplifting emotion that reflects happiness, satisfaction, and a sense of well-being. It often arises when our desires are fulfilled or we experience positive moments, and it helps energize

# 2. Baseline Inference

In [6]:
## Load Pretrained Model
pretrained_model_dir = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_dir)
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_dir, torch_dtype=torch.bfloat16
)
model.eval()
print("BASE MODEL LOADED")

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

BASE MODEL LOADED


## 2-1. Transformers

In [7]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Directly use generate() and tokenizer.decode() to get the output.
# Use `max_new_tokens` to control the maximum output length.
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)

In [8]:
print(tokenizer.decode(
    generated_ids[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
))

system
You are building an emotion‑prediction component that, given a character name and a brief situational description, must identify the character’s emotional state and the reason for it.
When given an input of the form:
```
Character:  {character}
Source: {source}
```
your job is to:

1. **Map the Emotion:**  
   - Choose among the 8 primary emotions from Plutchik’s Wheel (joy, trust, fear, surprise, sadness, disgust, anger, anticipation).  
   - For each, assign one of: `"na"` (not applicable), `"low"`, `"medium"`, or `"high"`.

2. **Write a Reason:**  
   - Provide a single sentence explaining why the character feels as you’ve labeled.

## Map the Emotion: Map the target emotion onto the 8 primary emotions from Plutchik’s Wheel
* "joy": Joy is a bright, uplifting emotion that reflects happiness, satisfaction, and a sense of well-being. It often arises when our desires are fulfilled or we experience positive moments, and it helps energize both our minds and bodies. Joy can enhance

## 2-2. Outlines

In [9]:
outlines_model = models.Transformers(model, tokenizer,)
generator = outlines.generate.json(outlines_model, EntryResult)

In [10]:
prediction = generator(text)

print("ANSWER:")
emotions = ["joy", "trust", "fear", "surprise", "sadness", "disgust", "anger", "anticipation"]
for emotion in emotions:
    print("{}: {}".format(emotion, row[emotion]))

print("PREDICTION:")
print(repr(prediction))
print(prediction.model_dump_json(indent=2))

ANSWER:
joy: na
trust: na
fear: low
surprise: na
sadness: high
disgust: na
anger: low
anticipation: medium
PREDICTION:
EntryResult(emotion=EmotionLabel(joy=<RelationshipStatus.na: 'na'>, trust=<RelationshipStatus.na: 'na'>, fear=<RelationshipStatus.low: 'low'>, surprise=<RelationshipStatus.na: 'na'>, sadness=<RelationshipStatus.high: 'high'>, disgust=<RelationshipStatus.na: 'na'>, anger=<RelationshipStatus.low: 'low'>, anticipation=<RelationshipStatus.medium: 'medium'>), reason='Michael feels overwhelmed and sad due to the numerous revisions needed and his doubts about completing the project on time.')
{
  "emotion": {
    "joy": "na",
    "trust": "na",
    "fear": "low",
    "surprise": "na",
    "sadness": "high",
    "disgust": "na",
    "anger": "low",
    "anticipation": "medium"
  },
  "reason": "Michael feels overwhelmed and sad due to the numerous revisions needed and his doubts about completing the project on time."
}


# 3. Tuned Version

In [12]:
run_name = "250418-01-qwen2_5-3b-try1"
run_name = "250421-01-qwen2_5-3b-mini-try1"
adapter_dir = f"weights/{run_name}/best"

model.load_adapter(adapter_dir)
model.eval()
print("MODEL LOADED")

MODEL LOADED


## 3-1. Transformers

In [13]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Directly use generate() and tokenizer.decode() to get the output.
# Use `max_new_tokens` to control the maximum output length.
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
)

In [14]:
print(tokenizer.decode(
    generated_ids[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True
))

system
You are building an emotion‑prediction component that, given a character name and a brief situational description, must identify the character’s emotional state and the reason for it.
When given an input of the form:
```
Character:  {character}
Source: {source}
```
your job is to:

1. **Map the Emotion:**  
   - Choose among the 8 primary emotions from Plutchik’s Wheel (joy, trust, fear, surprise, sadness, disgust, anger, anticipation).  
   - For each, assign one of: `"na"` (not applicable), `"low"`, `"medium"`, or `"high"`.

2. **Write a Reason:**  
   - Provide a single sentence explaining why the character feels as you’ve labeled.

## Map the Emotion: Map the target emotion onto the 8 primary emotions from Plutchik’s Wheel
* "joy": Joy is a bright, uplifting emotion that reflects happiness, satisfaction, and a sense of well-being. It often arises when our desires are fulfilled or we experience positive moments, and it helps energize both our minds and bodies. Joy can enhance

## 3-2. Outlines

In [15]:
outlines_model = models.Transformers(model, tokenizer,)

In [16]:
generator = outlines.generate.json(outlines_model, EntryResult)

In [17]:
print("original src: {}".format(row["original_src"]))
print("original tgt: {}".format(row["original_src"]))

pprint.pprint("Source: {}".format(row["source"]))
print("Character: {}".format(row["character"]))

original src: PersonX needs a lot of work
original tgt: PersonX needs a lot of work
('Source: Michael realizes his project proposal needs a lot of work before '
 'submission. He feels overwhelmed by the amount of revisions required and '
 'doubts his ability to finish on time. The pressure makes him feel '
 'downtrodden as he contemplates the long tasks ahead.')
Character: Michael


In [18]:
prediction = generator(text)

print("ANSWER:")
emotions = ["joy", "trust", "fear", "surprise", "sadness", "disgust", "anger", "anticipation"]
for emotion in emotions:
    print("{}: {}".format(emotion, row[emotion]))
print("reason: {}".format(row['reason']))

print("PREDICTION:")
print(repr(prediction))
print(prediction.model_dump_json(indent=2))

ANSWER:
joy: na
trust: na
fear: low
surprise: na
sadness: high
disgust: na
anger: low
anticipation: medium
reason: Michael feels downtrodden due to the heavy workload and looming pressure to improve his project.
PREDICTION:
EntryResult(emotion=EmotionLabel(joy=<RelationshipStatus.na: 'na'>, trust=<RelationshipStatus.na: 'na'>, fear=<RelationshipStatus.low: 'low'>, surprise=<RelationshipStatus.na: 'na'>, sadness=<RelationshipStatus.high: 'high'>, disgust=<RelationshipStatus.na: 'na'>, anger=<RelationshipStatus.low: 'low'>, anticipation=<RelationshipStatus.medium: 'medium'>), reason='Michael feels downtrodden because of the overwhelming amount of tasks and the doubts about completing the project on time.')
{
  "emotion": {
    "joy": "na",
    "trust": "na",
    "fear": "low",
    "surprise": "na",
    "sadness": "high",
    "disgust": "na",
    "anger": "low",
    "anticipation": "medium"
  },
  "reason": "Michael feels downtrodden because of the overwhelming amount of tasks and the dou